In [1]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import simpy

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import tensor_spec
#from env.RideSimulator.Grid import Grid
import tf_agents


import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from RideSimulator.taxi_sim import run_simulation


In [2]:
#register custom env
import gym

gym.envs.register(
     id='taxi-v0',
     entry_point='env.taxi:TaxiEnv',
     max_episode_steps=1500,
     kwargs={'state_dict':None},
)

In [3]:
#hyper params

num_iterations = 200 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}action

In [4]:
#load taxi env
env_name = "taxi-v0"
env = suite_gym.load(env_name)

tf_env = tf_py_environment.TFPyEnvironment(env)
reset = tf_env.reset()


In [11]:
#agent and policy
fc_layer_params = (100,)


q_net = q_network.QNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    fc_layer_params=fc_layer_params)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()


#random policy
policy = random_tf_policy.RandomTFPolicy(tf_env.time_step_spec(),
                                                tf_env.action_spec())


#replay buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=replay_buffer_max_length)
    


In [6]:
"""
policy.action(reset)
#tf_env.time_step_spec()
print(reset)
#print(env.reset())
#print(ts.restart(tf.convert_to_tensor(np.array([0,0,0,0], dtype=np.int32), dtype=tf.float32)))
print(" ")
print(ts.TimeStep(tf.constant([0]), tf.constant([0.0]), tf.constant([1.0]),tf.convert_to_tensor(np.array([[0,0,0,0]], dtype=np.int32), dtype=tf.float32)))

#print(tensor_spec.to_array_spec(reset))
#encoder_func = tf_agents.utils.example_encoding.get_example_encoder(env.reset())
#encoder_func(env.reset())
"""

'\npolicy.action(reset)\n#tf_env.time_step_spec()\nprint(reset)\n#print(env.reset())\n#print(ts.restart(tf.convert_to_tensor(np.array([0,0,0,0], dtype=np.int32), dtype=tf.float32)))\nprint(" ")\nprint(ts.TimeStep(tf.constant([0]), tf.constant([0.0]), tf.constant([1.0]),tf.convert_to_tensor(np.array([[0,0,0,0]], dtype=np.int32), dtype=tf.float32)))\n\n#print(tensor_spec.to_array_spec(reset))\n#encoder_func = tf_agents.utils.example_encoding.get_example_encoder(env.reset())\n#encoder_func(env.reset())\n'

In [7]:
r#un_simulation(policy)

[20000, 40000, 60000, 60000]
0 looking for driver at 0
0 looking for driver at 50
0 got 0 at 50
TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[10.,  0.,  0., 13.]], dtype=float32)>)
0 traveling to pickup point, [ 7 14] from [3 5]
driver 0 location updated  to  [51  9]
Trip distance: 44, time 60
0 finished the trip and released driver 0 at 104 to pool 9
Driver 0 finished trip
1 looking for driver at 200
1 looking for driver at 250
1 got 0 at 250
TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: sha

36 looking for driver at 7300
41 got 0 at 7300
TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[11., 17.,  0., 13.]], dtype=float32)>)
0 traveling to pickup point, [28 30] from [32 20]
driver 0 location updated  to  [31  0]
Trip distance: 30, time 7311
41 finished the trip and released driver 0 at 7341 to pool 6
Driver 0 finished trip
39 looking for driver at 7350
37 looking for driver at 7350
36 looking for driver at 7350
39 looking for driver at 7400
37 looking for driver at 7400
36 was cancelled!
39 looking for driver at 7450
37 looking for driver at 7450
39 looking for driver at 7500
37 looking for driver at 7500
42 looking for driver at 7550
39 looking for driver at 7550
37 looking for driver at 7550
43 looking for 

0 completed 34


[{'state': <RideSimulator.State.State at 0x7f7a846f00d0>,
  'action': PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>, state=(), info=()),
  'observation': [10, 0, 0, 13],
  'reward': 170},
 {'state': <RideSimulator.State.State at 0x7f79c8c4ea30>,
  'action': PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>, state=(), info=()),
  'observation': [12, 1, 0, 13],
  'reward': 110},
 {'state': <RideSimulator.State.State at 0x7f79c84020a0>,
  'action': PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>, state=(), info=()),
  'observation': [44, 2, 0, 13],
  'reward': 0},
 {'state': <RideSimulator.State.State at 0x7f79c84024f0>,
  'action': PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>, state=(), info=()),
  'observation': [20, 2, 0, 13],
  'reward': 0},
 {'state': <RideSimulator.State.State at 0x7f79c8402550>,
  'action': PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>, 

In [8]:
#ts.transition(np.array([1,2,3,4], dtype=np.int32), reward=0.0, discount=1.0)

TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([1, 2, 3, 4], dtype=int32))

In [9]:
#ts.termination(np.array([1,2,3,4], dtype=np.int32), reward=0.0)

TimeStep(step_type=array(2, dtype=int32), reward=array(0., dtype=float32), discount=array(0., dtype=float32), observation=array([1, 2, 3, 4], dtype=int32))

In [13]:
#train
num_iterations =10

for i in range (num_iterations):
    #run one episode of simulation and record states
    state_list = run_simulation(policy)
    states = []
    actions = []
    
    #convert states directly to tf timesteps
    for i in range(len(state_list)):
        #create time step
        if i == 0:
            #state_tf = ts.restart(np.array(state_list[i]["observation"], dtype=np.float32))
            state_tf = ts.TimeStep(tf.constant([0]), tf.constant([0.0]), tf.constant([1.0]), tf.convert_to_tensor(np.array([state_list[i]["observation"]], dtype=np.float32), dtype=tf.float32))
        if i == (len(state_list) - 1):
            state_tf = ts.TimeStep(tf.constant([1]), tf.constant(state_list[i]["reward"]), tf.constant([1.0]), tf.convert_to_tensor(np.array([state_list[i]["observation"]], dtype=np.float32), dtype=tf.float32))
            #state_tf = ts.termination(np.array(state_list[i]["observation"], dtype=np.float32), reward=state_list[i]["reward"])
        else:
            state_tf = ts.TimeStep(tf.constant([2]), tf.constant(state_list[i]["reward"]), tf.constant([0.0]), tf.convert_to_tensor(np.array([state_list[i]["observation"]], dtype=np.float32), dtype=tf.float32))
        
        #create action
        """if state_list[i]["action"] == 1:
            action = tf.constant([1], dtype=tf.int32)
        else:
            action = tf.constant([0], dtype=tf.int32)"""
        action = state_list[i]["action"]
            
        #print (state_tf)
        #print (action)
        states.append(state_tf)
        actions.append(action)
        
    for j in range(len(states)-1):
        present_state = states[j]
        next_state = states[j+1]
        action = actions[j]
        traj = trajectory.from_transition(present_state, action, next_state)
        
        # Add trajectory to the replay buffer
        buffer.add_batch(traj)
        #print(traj)
    
    
        
    """
    #re-register environemnt with new states
    env_name = 'taxi-v'+str(i)
    gym.envs.register(
         id=env_name,
         entry_point='env.taxi:TaxiEnv',
         max_episode_steps=1500,
         kwargs={'state_dict':state_list},
    )
    
    #reload new env
    env = suite_gym.load(env_name)
    tf_env = tf_py_environment.TFPyEnvironment(env)

    #reset tf env
    time_step = tf_env.reset()
    
    #loop through recorded steps
    for step in state_dict:
        present_state = tf_env.current_time_step()
        action = step.action
        new_state = tf_env.step(action)
        traj = trajectory.from_transition(time_step, action_step, next_time_step)
        replay_buffer.add_batch(traj)
    """
    
print(replay_buffer)

[20000, 40000, 60000, 60000]
0 looking for driver at 0
0 looking for driver at 50
0 got 0 at 50
0 looking for driver at 50
0 looking for driver at 100
1 looking for driver at 150
0 looking for driver at 150
1 looking for driver at 200
0 looking for driver at 200
1 looking for driver at 250
0 looking for driver at 250
1 got 0 at 250
1 looking for driver at 250
1 looking for driver at 300
0 looking for driver at 300
1 looking for driver at 350
0 looking for driver at 350
2 looking for driver at 400
1 looking for driver at 400
0 looking for driver at 400
2 looking for driver at 450
1 looking for driver at 450
0 looking for driver at 450
2 got 0 at 450
0 traveling to pickup point, [20 19] from [30  1]
driver 0 location updated  to  [3 6]
Trip distance: 21, time 471
2 finished the trip and released driver 0 at 492 to pool 0
Driver 0 finished trip
1 looking for driver at 500
0 looking for driver at 500
1 looking for driver at 550
0 looking for driver at 550
1 looking for driver at 600
0 look

41 looking for driver at 8400
45 got 0 at 8400
0 traveling to pickup point, [5 9] from [ 9 32]
driver 0 location updated  to  [37 10]
Trip distance: 32, time 8423
41 looking for driver at 8450
45 finished the trip and released driver 0 at 8455 to pool 6
Driver 0 finished trip
46 looking for driver at 8500
41 was cancelled!
47 looking for driver at 8550
46 looking for driver at 8550
47 looking for driver at 8600
46 looking for driver at 8600
46 got 0 at 8600
46 looking for driver at 8600
47 looking for driver at 8650
46 looking for driver at 8650
47 got 0 at 8650
0 traveling to pickup point, [59 31] from [37 10]
driver 0 location updated  to  [29 27]
Trip distance: 30, time 8680
46 looking for driver at 8700
47 finished the trip and released driver 0 at 8710 to pool 4
Driver 0 finished trip
48 looking for driver at 8750
46 looking for driver at 8750
48 looking for driver at 8800
46 looking for driver at 8800
48 got 0 at 8800
48 looking for driver at 8800
48 looking for driver at 8850
46

NameError: name 'buffer' is not defined

In [3]:
# startup simulation

def simpy_episode(rewards, steps, time_step, tf_env, policy):

    TIME_MULTIPLIER = 50
    DRIVER_COUNT = 1
    TRIP_COUNT = 8000
    RUN_TIME = 10000
    INTERVAL = 20
    # GRID_WIDTH = 3809
    # GRID_HEIGHT = 2622
    GRID_WIDTH = 60
    GRID_HEIGHT = 40
    HEX_AREA = 2.6

    Env = simpy.Environment()
    map_grid = Grid(env=Env, width=GRID_WIDTH, height=GRID_HEIGHT, interval=INTERVAL, num_drivers=DRIVER_COUNT,
                    hex_area=HEX_AREA)

    taxi_spots = map_grid.taxi_spots
    driver_list = create_drivers(Env, DRIVER_COUNT, map_grid)
    driver_pools = map_grid.driver_pools

    run_simulation(TRIP_COUNT, RUN_TIME, DRIVER_COUNT, TIME_MULTIPLIER, map_grid, taxi_spots, driver_list, driver_pools, Env, rewards, steps, time_step, tf_env, policy)
    t_count = 0
    for dr in driver_list:
        d_t_count = dr.total_trip_count
        t_count += d_t_count
        print(f"{dr.id} completed {d_t_count}")

    print(f"Total trip count: {t_count}")

In [5]:
var = tf.random.uniform([1], 0, 2, dtype=tf.int32)
var[0] = 2
print (var)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [10]:
#simple episode run - atttempt 1

time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
    simpy_episode(rewards, step, time_step, tf_env, policy)

    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 15)

In [ ]:
#simple episode run - atttempt 2

#time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
    time_step = tf_env.reset()
    
    
    
    
    
    
    simpy_episode(rewards, step, time_step, tf_env, policy)

    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

In [ ]:
#simple episode run template
"""
time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
  episode_reward = 0
  episode_steps = 0
  while not time_step.is_last():
    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

print('num_episodes:', num_episodes, 'num_steps:', num_steps)
print('avg_length', avg_length, 'avg_reward:', avg_reward)
"""